In [2]:
import sqlite3 
import pandas as pd 

In [3]:
df = pd.read_csv('dataset/customer_shopping_behavior.csv')

In [4]:
df.head(5)

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [5]:
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

In [6]:
df['Customer ID'].is_unique

True

In [7]:
conn = sqlite3.connect("inventory_custShop.db") 

In [8]:
tables = pd.read_sql_query("select name from sqlite_master where type ='table' " , conn)
tables

,name
0,custShoppingBehaviour


In [9]:
for table in tables['name']:
    print('-'*25,f'{table}','-'*25)
    print('count of records:',pd.read_sql(f"select count(*) as count from {table}",conn)['count'].values[0])
    display(pd.read_sql(f"select * from {table} limit 5 ",conn) ) 

------------------------- custShoppingBehaviour -------------------------
count of records: 3900


,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,age_group,frequency_purchase_days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,senior,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,young adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,adult,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,young adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,adult,365


Some Query Questions for better Analysis of this Behaviour Data 

Q1. what is the total revenue generated by male vs female customers ? 

In [10]:
query1 = f""" select gender,sum( purchase_amount) as total_Revenue from custShoppingBehaviour group by gender ; """
pd.read_sql_query(query1,conn)

,gender,total_Revenue
0,Female,75191
1,Male,157890


Q2. which Customers used a discount but still spent more than the average purchase amount ? 

In [11]:
pd.read_sql_query(f"select avg(purchase_amount) from custShoppingBehaviour ",conn)

,avg(purchase_amount)
0,59.764359


In [12]:
query2 = f""" select customer_id,purchase_amount from custShoppingBehaviour where discount_applied = 'Yes' and purchase_amount > (select avg(purchase_amount) from custShoppingBehaviour) """
pd.read_sql_query(query2,conn)

,customer_id,purchase_amount
0,2,64
1,3,73
2,4,90
3,7,85
4,9,97
...,...,...
834,1667,64
835,1671,73
836,1673,73
837,1674,62


Q3. which are the top 5 products with the highest average review rating ? 

In [13]:
query3 = f""" select item_purchased , round(avg(review_rating),3) as review_Raitng from custShoppingBehaviour group by (item_purchased) order by review_Rating desc limit 5  """
pd.read_sql_query(query3,conn)

,item_purchased,review_Raitng
0,Handbag,3.775
1,Boots,3.819
2,Dress,3.749
3,Backpack,3.752
4,T-shirt,3.778


Q4. Compare the average purchase amounts between standard and expresss shipping ? 

In [14]:
df['Shipping Type'].nunique()

6

In [15]:
query4 = f""" select round( avg(purchase_amount), 2 ) as avg_purchaseAmt, shipping_type from custShoppingBehaviour group by shipping_type having shipping_type IN ( "Express","Standard") """
pd.read_sql_query(query4,conn)

,avg_purchaseAmt,shipping_type
0,60.48,Express
1,58.46,Standard


Q5. Do subscribed customers spend more ? compare average spent and total revenue between subscribers     non-subscribers ? 

In [25]:
query5 = f""" select subscription_status,count(customer_id) as totalCustomer,round(avg(purchase_amount),2) as AvgPurchaseAmt , round(sum(purchase_amount),2) as TotalPurchaseAmount  from custShoppingBehaviour group by subscription_status order by TotalPurchaseAmount desc """
pd.read_sql_query(query5,conn)

,subscription_status,totalCustomer,AvgPurchaseAmt,TotalPurchaseAmount
0,No,2847,59.87,170436.0
1,Yes,1053,59.49,62645.0


Q6. which 5 products have the highest percentage of purchases with discounts applied ? 

In [19]:
query6 = f""" select item_purchased , ( round( count( case when discount_applied = "Yes" then 1 end)*100.0/count(*),2) ) as discount_purchased_item from custShoppingBehaviour group by item_purchased order by discount_purchased_item desc limit 5 """
pd.read_sql_query(query6,conn)

,item_purchased,discount_purchased_item
0,Hat,50.00
1,Sneakers,49.66
2,Coat,49.07
3,Sweater,48.17
4,Pants,47.37


Q7. Segment customers info New,returning , and loyal based on their total number of previous purchases, and show the count of each segment.

In [27]:
query7 = f""" with customer_type as( select customer_id,previous_purchases,case 
when previous_purchases = 1 then 'New'
when previous_purchases between 2 and 10 then 'Returning'
else 'Loyal'
end as customer_segment
from custShoppingBehaviour )
select count(customer_id),customer_segment
from customer_type
group by customer_segment
order by count(customer_id) desc 
"""
pd.read_sql_query(query7,conn)

,count(customer_id),customer_segment
0,3116,Loyal
1,701,Returning
2,83,New


Q8. What are the top 3 most purchased products with each category? 


In [22]:
query8 = f""" with ProdPurchased as (select item_purchased,category,count(*) as NoOfProductInCat from custShoppingBehaviour group by item_purchased,category ) select * from (select *,ROW_NUMBER() over( partition by category order by NoOfProductInCat desc) as rn  from ProdPurchased ) where rn <=3 """
pd.read_sql_query(query8,conn)

,item_purchased,category,NoOfProductInCat,rn
0,Jewelry,Accessories,171,1
1,Belt,Accessories,161,2
2,Sunglasses,Accessories,161,3
3,Blouse,Clothing,171,1
4,Pants,Clothing,171,2
5,Shirt,Clothing,169,3
6,Sandals,Footwear,160,1
7,Shoes,Footwear,150,2
8,Sneakers,Footwear,145,3
9,Jacket,Outerwear,163,1


Q9. find no of customers  who are repeat buyers ( more than 5 previous purchases) with grouping subscription status  ? 

In [29]:
query9 = f"""select  subscription_status , count(customer_id) as totalCustomer from custShoppingBehaviour where previous_purchases > 5 group by subscription_status """
pd.read_sql_query(query9,conn)

,subscription_status,totalCustomer
0,No,2518
1,Yes,958


Q10. What is the revenue contribution of each age group ? 

In [38]:
query10 = f""" select sum(purchase_amount),age_group from custShoppingBehaviour group by age_group """
pd.read_sql_query(query10,conn)

,sum(purchase_amount),age_group
0,110586,adult
1,69590,senior
2,52905,young adult
